In [13]:
import pandas as pd

sheet_url = "Wallet id.csv"
wallet_df = pd.read_csv(sheet_url)
wallet_df.head()


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [14]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [15]:
import requests
import time

API_KEY = "cqt_rQFmqpfdB9x6FHmHQ6rCJXpHcWMx"
wallets = wallet_df['wallet_id'].tolist()

def get_compound_tx(wallet):
    url = f"https://api.covalenthq.com/v1/1/address/{wallet}/transactions_v2/"
    params = {"key": API_KEY}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Collect data for all wallets
wallet_data = {}
for wallet in wallets:
    print(f"Fetching: {wallet}")
    data = get_compound_tx(wallet)
    if data:
        wallet_data[wallet] = data
    time.sleep(1)  # to avoid rate limiting


Fetching: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Fetching: 0x06b51c6882b27cb05e712185531c1f74996dd988
Fetching: 0x0795732aacc448030ef374374eaae57d2965c16c
Fetching: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Fetching: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Fetching: 0x104ae61d8d487ad689969a17807ddc338b445416
Fetching: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Fetching: 0x124853fecb522c57d9bd5c21231058696ca6d596
Fetching: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Fetching: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Fetching: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Fetching: 0x19df3e87f73c4aaf4809295561465b993e102668
Fetching: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Fetching: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
Fetching: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
Fetching: 0x22d7510588d90ed5a87e0f838391aaafa707c34b
Fetching: 0x24b3460622d835c56d9a4fe352966b9bdc6c20af
Fetching: 0x26750f1f4277221bdb5f6991473c6ece8c821f9d
Fetching: 0x27f72a000d8e9f324583f3a3491ea66998

In [25]:
features = []

for wallet, data in wallet_data.items():
    tx_list = data.get("data", {}).get("items", [])
    if not tx_list:
        continue

    # Example feature: number of transactions
    num_txns = len(tx_list)

    # You can extract more from tx_list[0] safely now
    first_tx = tx_list[0]
    gas_spent = int(first_tx.get("gas_spent", 0))
    value = int(first_tx.get("value", 0))

    features.append({
        "wallet": wallet,
        "num_txns": num_txns,
        "first_tx_gas_spent": gas_spent,
        "first_tx_value": value
    })

features_df = pd.DataFrame(features)
print(features_df.head())


                                       wallet  num_txns  first_tx_gas_spent  \
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3       100               55815   
1  0x06b51c6882b27cb05e712185531c1f74996dd988         5               21000   
2  0x0795732aacc448030ef374374eaae57d2965c16c         4              177613   

     first_tx_value  
0                 0  
1  5701850000000000  
2                 0  


In [27]:
def assign_score(row):
    score = 300  # base score

    # Reward for number of transactions
    score += min(row['num_txns'], 50) * 10

    # Reward for transaction value (scaled down)
    score += int(row['first_tx_value']) * 0.000000000001  # adjust scaling

    # Penalty for gas spent
    score -= row['first_tx_gas_spent'] * 0.001  # adjust penalty

    # Cap score between 0 and 1000
    return int(max(0, min(score, 1000)))

features_df['credit_score'] = features_df.apply(assign_score, axis=1)
print(features_df)


                                       wallet  num_txns  first_tx_gas_spent  \
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3       100               55815   
1  0x06b51c6882b27cb05e712185531c1f74996dd988         5               21000   
2  0x0795732aacc448030ef374374eaae57d2965c16c         4              177613   

     first_tx_value  credit_score  
0                 0           744  
1  5701850000000000          1000  
2                 0           162  


In [32]:
features = []

for wallet in wallets:
    print(f"Processing: {wallet}")
    data = wallet_data.get(wallet, None)
    tx_list = data.get("data", {}).get("items", []) if data else []

    if not tx_list:
        # Still store with 0/default values
        features.append({
            "wallet": wallet,
            "num_txns": 0,
            "avg_gas_spent": 0,
            "total_value_eth": 0,
            "max_tx_value_eth": 0
        })
        continue

    gas_spent = [int(tx["gas_spent"]) for tx in tx_list]
    value_eth = [int(tx["value"]) / 1e18 for tx in tx_list]

    features.append({
        "wallet": wallet,
        "num_txns": len(tx_list),
        "avg_gas_spent": sum(gas_spent) / len(gas_spent),
        "total_value_eth": sum(value_eth),
        "max_tx_value_eth": max(value_eth)
    })

features_df = pd.DataFrame(features)
print("Processed wallets:", features_df.shape[0])
features_df.to_csv("wallet_features.csv", index=False)


Processing: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Processing: 0x06b51c6882b27cb05e712185531c1f74996dd988
Processing: 0x0795732aacc448030ef374374eaae57d2965c16c
Processing: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Processing: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Processing: 0x104ae61d8d487ad689969a17807ddc338b445416
Processing: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Processing: 0x124853fecb522c57d9bd5c21231058696ca6d596
Processing: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Processing: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Processing: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Processing: 0x19df3e87f73c4aaf4809295561465b993e102668
Processing: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Processing: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
Processing: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
Processing: 0x22d7510588d90ed5a87e0f838391aaafa707c34b
Processing: 0x24b3460622d835c56d9a4fe352966b9bdc6c20af
Processing: 0x26750f1f4277221bdb5f6991473c6ece8c821f9d
Processing

In [40]:
for wallet, data in wallet_data.items():
    tx_list = data.get("data", {}).get("items", [])
    print(f"{wallet} → {len(tx_list)} txns")  # Add this
    if not tx_list:
        continue




0x0039f22efb07a647557c7c5d17854cfd6d489ef3 → 100 txns
0x06b51c6882b27cb05e712185531c1f74996dd988 → 5 txns
0x0795732aacc448030ef374374eaae57d2965c16c → 4 txns


In [36]:
def rule_based_score(row):
    score = 1000

    if row["num_txns"] < 5:
        score -= 300
    if row["total_value_eth"] < 0.01:
        score -= 200
    if row["avg_gas_spent"] < 25000:
        score -= 100

    return max(score, 0)

features_df["score"] = features_df.apply(rule_based_score, axis=1)
features_df[["wallet", "score"]].to_csv("wallet_risk_scores.csv", index=False)


In [38]:
features_df[["wallet", "score"]].to_csv("rule_based_wallet_risk_scores.csv", index=False)


In [39]:
print(features_df[["wallet", "score"]].head())
print(features_df[["wallet", "score"]].shape)


                                       wallet  score
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3   1000
1  0x06b51c6882b27cb05e712185531c1f74996dd988   1000
2  0x0795732aacc448030ef374374eaae57d2965c16c    700
(3, 2)
